#Question 1 

Question 1: Use dataset of SET50 list in 2020 (https://www.set.or.th/th/market/files/constituents/SET50_100_H2_2020_revised.pdf) Compute the return in 2020 from 2020-01-01 to 2020-12-31. Find out

Average annual return _____ and annual STD __ for buy and hold


Average annual return _____ and annual STD __ for moving average crossing


Average annual return _____ and annual STD __ for Bollinger Band


Average annual return _____ and annual STD __ for RSI


In [ ]:
import csv 
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
def buyAndHold(data_frame, is_plot=False):
  """
  Buy and Hold strategy:
  Inputs: 
          data_frame: pandas dataframe of the stock prices. It must have the "Adj Close" prices
          is_plot: If true, this code will plot the Adj close price and show the buy and sell points.
  output:
          signal indicator: the series of indicators 1 to take the position, 0 for keep previous status and -1 to clear the position.

  """

  close = data_frame['Adj Close']
  status = 0
  signal = []
  for k in range(close.shape[0]):
    if k == 0:
      signal.append(1)
    else:
      signal.append(0)
  signal = pd.Series(data=signal, index=data_frame.index)
  buy_sig = close[signal == 1]
  sell_sig = close[signal == -1]
  if is_plot:
    plt.figure(figsize=(15,10))
    plt.plot(close)    
    plt.plot(buy_sig, color="red", marker="^",linestyle="")   
    plt.plot(sell_sig, color="black", marker="v",linestyle="")  
    plt.legend(["Adj Close","Buy", "Sell"])
    plt.title("Buy and Hold")
    plt.grid()
  return signal


In [ ]:
def movingavg(data_frame, is_plot=False):
  """
  Moving Average crossing strategy:
  Inputs: 
          data_frame: pandas dataframe of the stock prices. It must have the "Adj Close" prices
          is_plot: If true, this code will plot the Adj close price and show the buy and sell points.
  output:
          signal indicator: the series of indicators 1 to take the position, 0 for keep previous status and -1 to clear the position.

  """
  long_win = 25
  short_win = 10
  close = data_frame['Adj Close'] 
  mu_long = close.rolling(long_win).mean()
  mu_short = close.rolling(short_win).mean()
  signal = []
  status = 0
  for k in range(close.shape[0]):
    if ( mu_short[k] > mu_long[k]) and (status ==0):
      signal.append(1)
      status = 1
    elif (mu_short[k] < mu_long[k]) and (status ==1):
      signal.append(-1)
      status = 0
    else:
      signal.append(0)

  signal = pd.Series(data=signal, index=data_frame.index)
  buy_sig = close[signal == 1]
  sell_sig = close[signal == -1]
  if is_plot:
    plt.figure(figsize=(15,10))
    plt.plot(close)    
    plt.plot(mu_long)
    plt.plot(mu_short)
    plt.plot(buy_sig, color="red", marker="^",linestyle="")   
    plt.plot(sell_sig, color="black", marker="v",linestyle="")  
    plt.legend(["Adj Close",f"MA({long_win})", f"MA({short_win})", "Buy", "Sell"])
    plt.title("Moving Average Crossing")
    plt.grid()
  return signal


In [ ]:
def bollinger(data_frame, is_plot=False):
  """
  Bollinger Band strategy:
  Inputs: 
          data_frame: pandas dataframe of the stock prices. It must have the "Adj Close" prices
          is_plot: If true, this code will plot the Adj close price and show the buy and sell points.
  output:
          signal indicator: the series of indicators 1 to take the position, 0 for keep previous status and -1 to clear the position.

  """
  close = data_frame['Adj Close']
  win = 10 
  ma = close.rolling(win).mean()
  std = close.rolling(win).std()
  up = ma + std
  low = ma - std  
  status = 0
  signal = []
  for k in range(close.shape[0]):
    closek = close[k]
    if (closek >  up[k]) and (status == 0): # buy
      signal.append(1)
      status = 1
    elif (closek < low[k]) and (status == 1): # sell
      signal.append(-1)
      status = 0
    else:
      signal.append(0)

  signal = pd.Series(data=signal, index=data_frame.index)
  buy_sig = close[signal == 1]
  sell_sig = close[signal == -1]
  if is_plot:
    plt.figure(figsize=(15,10))
    plt.plot(close)
    plt.plot(up)
    plt.plot(low) 
    plt.plot(buy_sig, color="red", marker="^",linestyle="")   
    plt.plot(sell_sig, color="black", marker="v",linestyle="")  
    plt.legend(["Adj Close","High", "Low", "Buy", "Sell"])
    plt.title("Bollinger Bands")
    plt.grid()
  return signal


In [ ]:
def rsi(data_frame, is_plot=False):
  """
  Relative Signal Strength:
  Inputs: 
          data_frame: pandas dataframe of the stock prices. It must have the "Adj Close" prices
          is_plot: If true, this code will plot the Adj close price and show the buy and sell points.
  output:
          signal indicator: the series of indicators 1 to take the position, 0 for keep previous status and -1 to clear the position.

  """
  window_length= 14
  upv = 70
  lowv= 40
  close = data_frame['Adj Close']
  delta = close.diff(1) 
 
  up, down = delta.copy(), delta.copy()
  up[up < 0] = 0
  down[down > 0] = 0
  roll_up1 = up.ewm(span=window_length).mean()
  roll_down1 = down.abs().ewm(span=window_length).mean()
  # Calculate the RSI based on EWMA
  RS1 = roll_up1 / roll_down1
  RSI1 = 100.0 - (100.0 / (1.0 + RS1))
  status = 0
  signal = []
  for k in range(close.shape[0]):
    rsk = RSI1[k]
    if (rsk < lowv) and (status == 0): # buy
      signal.append(1)
      status = 1
    elif (rsk > upv) and (status == 1): # sell
      signal.append(-1)
      status = 0
    else:
      signal.append(0)

  signal = pd.Series(data=signal, index=data_frame.index)
  buy_sig = close[signal == 1]
  sell_sig = close[signal == -1]

  if is_plot:  
    plt.figure(figsize=(15,10))
    plt.subplot(2,1,1)
    plt.plot(close)
    plt.plot(buy_sig, color="red", marker="^",linestyle="")   
    plt.plot(sell_sig, color="black", marker="v",linestyle="")  
    plt.legend(["Adj Close","Buy", "Sell"])
    plt.grid()
    plt.title("RSI")
    plt.subplot(2,1,2)
    plt.plot(RSI1)
    plt.plot(RSI1.index, upv*np.ones((close.shape[0],)),"b--")
    plt.plot(RSI1.index, lowv*np.ones((close.shape[0],)),"b--")
    plt.legend(['RSI', "SELL Band", "BUY Band"])
    plt.grid()
    plt.title("RSI Value")
  return signal


In [ ]:
name = ["ADVANC","AOT","AWC","BBL","BDMS","BEM","BGRIM","BH","BJC","BPP","BTS",
        "CBG","CPALL","CPF","CPN","CRC","DTAC","EA","EGCO","GLOBAL","GPSC","GULF","HMPRO","INTUCH",
        "IRPC","IVL","KBANK","KTB","KTC","LH","MINT","MTC","OSP","PTT","PTTEP","PTTGC","RATCH","SAWAD",
        "SCB","SCC","SCGP","TISCO","TMB","TOA","TOP","TRUE","TTW","TU","VGI","WHA"]
set50 = dict()
for stock in name:
  dataframe = data.DataReader(f"{stock}.bk",
  data_source="yahoo", 
  start='2020-01-01', 
  end='2020-12-31')
  ret = dataframe['Adj Close'].pct_change(1).dropna()
  set50[stock] = ret

set50 = pd.DataFrame(set50)
print(set50.cov())

          ADVANC       AOT       AWC  ...        TU       VGI       WHA
ADVANC  0.000284  0.000247  0.000313  ...  0.000144  0.000212  0.000242
AOT     0.000247  0.000997  0.000899  ...  0.000159  0.000641  0.000625
AWC     0.000313  0.000899  0.001677  ...  0.000208  0.000879  0.000898
BBL     0.000148  0.000539  0.000638  ...  0.000163  0.000457  0.000504
BDMS    0.000198  0.000540  0.000666  ...  0.000147  0.000439  0.000482
BEM     0.000221  0.000548  0.000651  ...  0.000187  0.000557  0.000540
BGRIM   0.000285  0.000550  0.000643  ...  0.000254  0.000570  0.000578
BH      0.000180  0.000374  0.000522  ...  0.000221  0.000320  0.000330
BJC     0.000257  0.000478  0.000613  ...  0.000285  0.000461  0.000534
BPP     0.000214  0.000471  0.000648  ...  0.000177  0.000580  0.000571
BTS     0.000232  0.000602  0.000710  ...  0.000172  0.000607  0.000550
CBG     0.000261  0.000474  0.000595  ...  0.000266  0.000652  0.000592
CPALL   0.000123  0.000247  0.000336  ...  0.000105  0.000251  0

In [ ]:
def techicalTrading(stock_info, # dataframe of a stock
                    indicator, # trade indicator
                    initial_money=1000000 #Starting money
                    ):
  """
  Trader simulation:
  Inputs: 
        stock_info: stock price data frames
        indicator: time-series indicator: 1 for Buy, -1 for sell, 0 for nothing
        initinal_money: initial money in the port. default 1M 
  output:
        wealth: time-series money in the port
  """

  num_dates, _ = stock_info.shape
  status = "NONE" # Start with no poistion
  stock= 0
  money = initial_money
  wealth = []
  close = stock_info['Adj Close']
  for k in range(num_dates):
    indicatork = indicator[k]
    if not np.isnan(indicatork): # if it is not NA
      if (indicatork > 0) and ( status == "NONE"):
        stock = money/close.values[k]
        #print(f"Buy at {close.values[k]}")
        money = 0
        status = "HOLD"
      elif (indicatork < 0) and ( status == "HOLD"):
        money = stock * close.values[k]
        stock =  0
        #print(f"Sell at {close.values[k]}")
        status = "NONE"
    cur_wealth = money + stock * close.values[k]
    wealth.append(cur_wealth)
  wealth = pd.Series(data=wealth, index=stock_info.index)  
  return wealth

Buy And Hold

In [ ]:
dataframe = data.DataReader("ADVANC.BK", 
data_source ="yahoo",
start = "2020-01-01",
end =  "2020-12-31") 
plt.figure(figsize=(15,15))
indicator = buyAndHold(dataframe, is_plot=False)
print(f"Average annual return is {indicator.mean()*100.0:2.2f}%")
print(f"Average annual STD is {indicator.std()*100.0:2.2f}")

Average annual return is 0.41%
Average annual STD is 6.42


<Figure size 1080x1080 with 0 Axes>

Moving Average Crossing

In [ ]:
dataframe = data.DataReader("ADVANC.BK", 
data_source ="yahoo",
start = "2020-01-01",
end =  "2020-12-31") 
plt.figure(figsize=(15,15))
indicator = movingavg(dataframe, is_plot=False)
print(f"Average annual return is {indicator.mean()*100.0:2.2f}%")
print(f"Average annual STD is {indicator.std()*100.0:2.2f}")

Average annual return is 0.00%
Average annual STD is 24.05


<Figure size 1080x1080 with 0 Axes>

 Bollinger Band

In [ ]:
dataframe = data.DataReader("ADVANC.BK", 
data_source ="yahoo",
start = "2020-01-01",
end =  "2020-12-31") 
plt.figure(figsize=(15,15))
indicator = bollinger(dataframe, is_plot=False)
print(f"Average annual return is {indicator.mean()*100.0:2.2f}%")
print(f"Average annual STD is {indicator.std()*100.0:2.2f}")

Average annual return is 0.00%
Average annual STD is 30.15


<Figure size 1080x1080 with 0 Axes>

RSI

In [ ]:
dataframe = data.DataReader("ADVANC.BK", 
data_source ="yahoo",
start = "2020-01-01",
end =  "2020-12-31") 
plt.figure(figsize=(15,15))
indicator = rsi(dataframe, is_plot=False)
print(f"Average annual return is {indicator.mean()*100.0:2.2f}%")
print(f"Average annual STD is {indicator.std()*100.0:2.2f}")

Average annual return is 0.41%
Average annual STD is 21.32


<Figure size 1080x1080 with 0 Axes>

# Question 2

Question 2: Compute 95% confidence interval of the average annual return for each strategies.

1.Specify the type of statistics used in computing the confidence interval ___z-statistics (z-statistics or t-statistics)


2..95% Confidence interval for Buy and hold is _____.


3.95% Confidence interval for moving average crossing is _____.


4.95% Confidence interval for Bollinger Band is _____.


5.95% Confidence interval for RSI is _____.

In [ ]:
import scipy.stats as st
from scipy.stats import norm

In [ ]:
indicator = buyAndHold(dataframe, is_plot=False)

ret = indicator.pct_change(1).dropna() 
num = indicator.count()
mean = indicator.mean()
std = indicator.std()

Interval = st.norm.interval(0.95, loc=mean, scale=std/np.sqrt(num))
print(f"95% Confidence interval for Buy and hold is {Interval}")